In [1]:
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# data augmentation 미적용 , 에폭이 30 이상일 경우 early stopping 포함, 
# 에폭 : 10, , dropout_rate: 0.5 , batch size : 16 , data augmentation 미적용 , early stopping 미포함, 
# 에폭 : 30, dropout_rate: 0.5, batch size: 32, # data augmentation 미적용 , early stopping 포함,  (에폭을 30으로 증가시켰으므로)
# ================================
# 1. 라이브러리 및 시드 설정
# ================================
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# 시드 고정 함수
def set_seed(seed=123):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    print("시드 고정 완료.")

set_seed(123)

# ================================
# 2. 하이퍼파라미터 설정
# ================================
IMG_SIZE = 180
BATCH_SIZE = 16       # 🔧 수정 가능
EPOCHS = 10           # 🔧 30 이상이면 EarlyStopping 적용됨
LEARNING_RATE = 1e-5
DENSE_UNITS = 256
DROPOUT_RATE = 0.5

# ================================
# 3. 데이터셋 로딩 및 전처리
# ================================
(raw_train, raw_val, raw_test), info = tfds.load(
    'cassava',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
print("클래스 수:", num_classes)

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

train_ds = raw_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(1)
val_ds = raw_val.map(preprocess).batch(BATCH_SIZE).prefetch(1)
test_ds = raw_test.map(preprocess).batch(BATCH_SIZE).prefetch(1)

# ================================
# 4. 모델 구성 (Data Augmentation ❌)
# ================================
conv_base = keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(DENSE_UNITS, activation='relu')(x)
x = layers.Dropout(DROPOUT_RATE)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.RMSprop(learning_rate=LEARNING_RATE),
    metrics=["accuracy"]
)

model.summary()

# ================================
# 5. 학습 (조건부 EarlyStopping 포함)
# ================================
callbacks = [
    keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss")
]

if EPOCHS >= 30:
    print(f"✅ 에폭이 {EPOCHS}이므로 EarlyStopping을 진행합니다.")
    callbacks.append(
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=7, # 기존 5 → 7로 변경하여 성능 저하를 한두 번은 허용
            restore_best_weights=True
        )
    )
else:
    print(f"⚠️ 에폭이 {EPOCHS}이하이므로 EarlyStopping을 진행하지 않습니다.")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

# ================================
# 6. 평가
# ================================
model = keras.models.load_model("best_model.h5")
test_loss, test_acc = model.evaluate(test_ds)
print(f"테스트 정확도: {test_acc:.3f}")

# ================================
# 7. 학습 곡선 시각화
# ================================
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(acc) + 1)

plt.plot(epochs_range, acc, 'bo', label='Train Acc')
plt.plot(epochs_range, val_acc, 'b', label='Val Acc')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs_range, loss, 'bo', label='Train Loss')
plt.plot(epochs_range, val_loss, 'b', label='Val Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# ================================
# 8. Confusion Matrix & Report
# ================================
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred))
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

#-------------------------------- ROC 커브 추가

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle

# 클래스 수에 맞게 이진화
y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))
y_score = []

# 예측 확률 수집
for images, _ in test_ds:
    preds = model.predict(images)
    y_score.extend(preds)

y_score = np.array(y_score)

# ROC Curve 계산
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# 마이크로 평균 ROC
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot
plt.figure(figsize=(10, 7))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])

for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='Class {0} (AUC = {1:0.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


2025-06-06 11:11:41.107076: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-06 11:11:41.109619: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 11:11:41.139967: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 11:11:41.141681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-06 11:11:41.643841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

시드 고정 완료.


2025-06-06 11:11:43.256402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-06 11:11:43.278087: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


클래스 수: 5
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 180, 180, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 180, 180, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 5, 5, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 256)               3277056   
                                                    

2025-06-06 11:11:43.771892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [8]
	 [[{{node Placeholder/_1}}]]
2025-06-06 11:11:43.772529: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [8]
	 [[{{node Placeholder/_2}}]]


 39/354 [==>...........................] - ETA: 3:16 - loss: 4.9993 - accuracy: 0.3141

KeyboardInterrupt: 